In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

# Project 2 – COVID Vaccinations 🦠

## DSC 80, Winter 2023

### Checkpoint Due Date: Thursday, April 27th (Questions 1, 2, 6, 8, and 10)
### Due Date: Thursday, May 4th

## Instructions

---

### Working on the Project

This Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems.

* Like the lab, your coding work will be developed in the accompanying `project.py` file, that will be imported into the current notebook. This code will be autograded.
* There is no manually-graded component to Project 2, so the only thing you will ever submit is `project.py`.
* **For the Checkpoint, which is required, you only need to turn in a `project.py` containing solutions for Questions 1, 2, 6, 8, and 10!**
    - The "Project 2 Checkpoint" autograder on Gradescope does not thoroughly check your code – it only runs the public tests on Questions 1, 2, 6, 8, and 10 to make sure that you have completed them. There are no hidden tests for the checkpoint, and you will see your score upon submission. 
    - When you submit the final version of the project, however, we will use hidden tests to check your answers more thoroughly.
    - Note that this means you will ultimately have to submit the project twice – once to the "Project 2 Checkpoint" autograder (Questions 1, 2, 6, 8, and 10) and once to the "Project 2" autograder (once you're fully done).
- **Do not change the function names in the `project.py` file!** The functions in the `project.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2023-wi).
- **To ensure that all of your work to be submitted is in `project.py`, we've provided an additional uneditable notebook, called `project-validation.ipynb`, that contains only the tests and their setup. Make sure you are able to run it top-to-bottom without error before submitting!**
- You are encouraged to write your own additional helper functions to solve the project, as long as they also end up in `project.py`.

### Warning! 🚨

This project contains larger datasets than usual. They aren't huge, but they aren't tiny. The autograder has been set to allow more memory usage than usual, but if you write inefficient code you may exhaust all of the allocated memory, causing the autograder to fail. If this happens, the autograder will say that your submission was formatted incorrectly.

**Writing efficient code is part of the project!** Be sure to submit your code to the Gradescope autograder regularly to check that it is efficient enough. If your code fails to run on the autograder due to efficiency reasons and it is past the late deadline, we will perform a "catastrophic regrade" by removing your inefficient code (deleting that function definition) and charging the usual cost of two slip days. This is expensive, so be sure to check that your code works with the autograder with plenty of time before the deadline!


### Working with a Partner 👯

You may work together on projects (and projects only!) with a partner. If you work with a partner, you are both required to actively contribute to all parts of the project. You must both be working on the assignment at the same time together, either physically or virtually on a Zoom call. You are encouraged to follow the pair programming model, in which you work on just a single computer and alternate who writes the code and who thinks about the problems at a high level.

In particular, you **cannot** split up the project and each work on separate parts independently.

Note that if you do work with a partner, you and your partner must submit the Checkpoint together and the whole project together. See [here](https://dsc80.com/syllabus/#projects) for more details.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import os

from IPython.display import clear_output

import pathlib
import urllib.request

In [4]:
from project import *

In addition to the above usual imports, we'll need a few packages that aren't included in the standard `dsc80` conda environment we had you configure. 

Before running the cell below for the first time, run the following lines in your Terminal after activating your `dsc80` conda environment, and then restart your kernel. If you've already installed these packages, you should be able to run this cell without error.

```
pip install ipywidgets
jupyter nbextension enable --py --sys-prefix widgetsnbextension
pip install plotly
```

In [5]:
from ipywidgets import widgets, interact, FloatSlider
import plotly.express as px

## About the Assignment 🧐

Data science has been an important tool in understanding the development of the COVID-19 pandemic. In this project, we'll take a took at two real-world datasets that measure vaccination rates and effectiveness.

---

<a name='outline'></a>

### Navigating the Project 🗺

Click on the links below to navigate to different parts of the project. Note that Questions 1, 2, 6, 8, and 10 constitute your Checkpoint submission.

- [Part 1: Global Vaccination Rates 🌎](#part1)
    - [Question 1 (✅ Checkpoint Question)](#Question-1)
    - [Question 2 (✅ Checkpoint Question)](#Question-2)
    - [Question 3](#Question-3)
    - [Question 4](#Question-4)
    - [Question 5](#Question-5)
- [Part 2: Vaccine Effectiveness 💉](#part2)
    - [Question 6 (✅ Checkpoint Question)](#Question-6)
    - [Question 7](#Question-7)
    - [Question 8 (✅ Checkpoint Question)](#Question-8)
    - [Question 9](#Question-9)
    - [Question 10 (✅ Checkpoint Question)](#Question-10)
    - [Question 11](#Question-11)

---

The file `data/covid_vaccinations.csv` contains information on the number of COVID vaccinations that have been administered in various countries and regions of the world, as of January 20th, 2023:

In [6]:
vacs = pd.read_csv(os.path.join('data', 'covid_vaccinations.csv'))
vacs

,Date,Country_Region,Doses_admin,People_at_least_one_dose
0,2020-12-29,Austria,2.123000e+03,2123
1,2020-12-29,Bahrain,5.501400e+04,55014
2,2020-12-29,Belarus,0.000000e+00,0
3,2020-12-29,Belgium,3.400000e+02,340
4,2020-12-29,Canada,5.907900e+04,59078
...,...,...,...,...
133517,2023-01-20,West Bank and Gaza,3.748571e+06,2012767
133518,2023-01-20,World,1.325698e+10,5514130774
133519,2023-01-20,Yemen,1.242982e+06,1006274
133520,2023-01-20,Zambia,1.361498e+07,10959569


<a name='part1'></a>

## Part 1: Global Vaccination Rates 🌎

One question we might hope to answer with this data is, **in each country, what is the average number of doses received?** In countries that have been more proactive about booster rollouts, you might expect this number to be much higher than 2. 

<br>

<center><b>What do you think this number is for the US?</b></center>

Eventually, to answer this question, you will create a **[choropleth](https://en.wikipedia.org/wiki/Choropleth_map)**, like the one below.

<img src='data/choropleth-example.png' width=70%>

We may think that we can use this data directly for getting the vaccination rate but unfortunately, the messiness of real-world data means that answering this question requires a bit of cleaning and organization. We'll do this cleaning and organization in the next few questions.

First, we need to find the total number of vaccinated individuals in each country/region. To do this, we have to understand what the values in the `'Doses_admin'` column mean. Are they the number people vaccinated on *that day*? Or are they the *cumulative* number of vaccinated people? This can be answered with a plot. Run the cell below to display an ✨ **interactive** ✨ plot that displays the `'Doses_admin'` column for any country you select.

In [7]:
DEFAULT = 'US'

def plot_cases(country):
    country_only = vacs[vacs['Country_Region'] == country]
    fig = px.line(country_only, x='Date', y='Doses_admin', title=f"'Doses_admin' column for {country}")
    fig.show()
    

dropdown_cases = widgets.Dropdown(options=np.sort(vacs['Country_Region'].unique()), value=DEFAULT)

def dropdown_cases_handler(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        clear_output()
        display(dropdown_cases)
        plot_cases(change['new'])
        
display(dropdown_cases)
plot_cases(DEFAULT)
dropdown_cases.observe(dropdown_cases_handler)

Dropdown(index=179, options=('Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', …

Click the drop down menu to change the country being visualized. For many countries, it appears that the `'Doses_admin'` column is strictly increasing. 

However, that's not the case for all countries – **for instance, select Samoa.** Notice the dip in the graph? Unless there were *negative* doses administered (which would be... awkward), we must conclude that the data is messy - there will be instances where, either due to errors in reporting or in recording, the cumulative number of doses actually *decreases* from one day to the next.

### Question 1 (✅ Checkpoint Question)

<a name='Question-1'></a>

([return to the outline](#outline))

Let's determine how common the messiness we saw above for Samoa is across all countries.

#### `count_monotonic`

Complete the implementation of the function `count_monotonic`, which takes in a 1D `numpy` array and returns the number of times an entry in the array (except the first) is **strictly less** than the previous entry. Example behavior is given below.

```py
>>> count_monotonic(np.array([3, 6, 6, 2, 5, 8]))
1

>>> count_monotonic(np.array([4.5, 5.2, -1.2, -2.0, 4.9, -2.0]))
3
```

***Hint***: There's a `numpy` function for computing the difference between consecutive elements of an array.

<br>

#### `monotonic_violations_by_country`

Complete the implementation of the function `monotonic_violations_by_country`, which takes in a DataFrame like `vacs` and returns a DataFrame with one row per country and two `int` columns – `'Doses_admin_monotonic'` and `'People_at_least_one_dose_monotonic'`. An entry in the `'Doses_admin_monotonic'` column should be the number of times a country's `'Doses_admin'` is NOT monotonically increasing; likewise for the other columns. The index of the returned DataFrame should contain country names, and the order of the two columns in your returned DataFrame does not matter.

***Note:*** No looping is allowed, but `groupby` is your friend. Use the `count_monotonic` function, along with the assumption that the input DataFrame `vacs` is already sorted by date.

In [8]:
# run this to see the result, and don't change this cell -- it is used by the tests
mono = monotonic_violations_by_country(vacs)
mono

,Doses_admin_monotonic,People_at_least_one_dose_monotonic
Country_Region,,
Afghanistan,0,0
Albania,0,0
Algeria,0,0
Andorra,0,0
Angola,0,0
...,...,...
West Bank and Gaza,0,0
World,2,6
Yemen,1,0


In [9]:
count_monotonic(np.array([4.5, 5.2, -1.2, -2.0, 4.9, -2.0]))

3

In [10]:
grader.check("q1")

q1 results: All test cases passed!

If you've completed Question 1 correctly, you should see that it is not rare for the data to violate the monotonicity assumption (and multiple times at that). We'll have to take this into account when doing our data analysis.

### Question 2 (✅ Checkpoint Question)

<a name='Question-2'></a>

([return to the outline](#outline))

To get the average number of doses recieved, we'd like to get the total number of doses administered and people vaccinated for each country. If the data were truly monotonic, we could simply group by country and find the maximum within each group and within each column – this would be the overall cumulative total. However, if the data has errors that make it not monotonically increasing, there could be a noisy "spike" in the graph, as you saw in the plot for Samoa, so that the maximum may be an *unfaithful* measure of the total number of doses. Instead of the maximum, we should use something more robust, such as **the 97th percentile**.

Complete the implementation of the function `robust_totals`, which takes in a DataFrame like `vacs` and returns a DataFrame with one row for each country and two columns – `'Doses_admin'` and `'People_at_least_one_dose'`. The `'Doses_admin'` column should contain the 97th percentile of the `'Doses_admin'` values for each country; likewise for the other column. The index of the returned DataFrame should contain country names, and the order of the two columns in your returned DataFrame does not matter.

In [11]:
# run this to see the result, and don't change this cell -- it is used by the tests
tots = robust_totals(vacs)
tots

,Doses_admin,People_at_least_one_dose
Country_Region,,
Afghanistan,1.244987e+07,1.145969e+07
Albania,3.025728e+06,1.342243e+06
Algeria,1.526744e+07,7.840131e+06
Andorra,1.543670e+05,5.790100e+04
Angola,2.370105e+07,1.499013e+07
...,...,...
West Bank and Gaza,3.748571e+06,2.012767e+06
World,1.319737e+10,5.503074e+09
Yemen,1.239152e+06,1.003208e+06


In [12]:
grader.check("q2")

q2 results: All test cases passed!

Recall that we want to calculate the average number of doses received per person in a country. However, to get this value we also need information on the total population of a country. Luckily, we have access to a separate dataset of country population statistics:

In [13]:
pops_raw = pd.read_csv(os.path.join('data', 'populations.csv'))
pops_raw

,Country (or dependency),Population in 2023,World Percentage,Area (Km²),Density (P/Km²),ISO
0,India,1428627.663,17.85%,3_287_590 Km²,480.5033/Km²,IND
1,China,1425671.352,17.81%,"9,706,961 Km²",151.2696/Km²,CHN
2,United States,339996.563,4.25%,"9,372,610 Km²",37.1686/Km²,USA
3,Indonesia,277534.122,3.47%,"1,904,569 Km²",147.8196/Km²,IDN
4,Pakistan,240485.658,3.00%,881_912 Km²,311.9625/Km²,PAK
...,...,...,...,...,...,...
200,San Marino,33.642,0.00%,61 Km²,560.7000/Km²,SMR
201,Palau,18.058,0.00%,459 Km²,39.2565/Km²,PLW
202,Nauru,12.780,0.00%,21 Km²,639.0000/Km²,NRU
203,Tuvalu,11.396,0.00%,26 Km²,379.8667/Km²,TUV


### Question 3

([return to the outline](#outline))

Unfortunately, this DataFrame's data types require some cleaning. All of the columns except the first and last are numeric and should either be `int`s or `float`s, as appropriate, but many of them are being stored incorrectly.

Complete the implementation of the function `fix_dtypes` which takes in a DataFrame like `pops_raw` above and returns a DataFrame with exactly the same columns and rows, but with the `dtype`s "fixed" to be appropriate for the data contained within.

***Notes:***
 - All percentages should be represented as decimals – e.g., 27% should be 0.27.
 - `'Population in 2023'` is currently being stored in thousands. That is, the population of Tokelau is 11,396 people, but it is represented as 11.396 in the raw dataset. "Convert" this population total to the actual number of people. In your resulting DataFrame, the population of Tokelau should be 11396.
 - Store each column using the most appropriate `dtype` after cleaning – for instance, don’t store values in a column as `float`s if they’re all whole numbers.
 - You can make helper functions! It's OK to loop over the **columns** of the DataFrame (but not the rows).

As always, make sure that your function does not change the DataFrame it is called on!

In [15]:
pops_raw.dtypes

Country (or dependency)     object
Population in 2023         float64
World Percentage            object
Area (Km²)                  object
Density (P/Km²)             object
ISO                         object
dtype: object

In [25]:
# run this to see the result, and don't change this cell -- it is used by the tests
pops = fix_dtypes(pops_raw)
pops.dtypes

Country (or dependency)     object
Population in 2023           int32
World Percentage           float64
Area (Km²)                   int64
Density (P/Km²)            float64
ISO                         object
dtype: object

In [26]:
grader.check("q3")

q3 results: All test cases passed!

### Question 4

([return to the outline](#outline))

The next step in solving our initial question is to take the population values from the previous problem and insert them into the DataFrame with our vaccination data, `tots`. Typically this can be done with a simple `concat` or `merge`, but there's a problem: the names of the countries are not always the same in each DataFrame.

#### `missing_in_pops`
First, complete the implementation of the function `missing_in_pops` which takes in two DataFrames, the first like `tots` above, containing the total number of vaccinations per country, and the second like `pops` above, containing the population of each country. It should return a Python **set** of country names that appear in `tots` but not in `pops`.

***Hint:*** Your solution should only take a few lines; don't use a `for`-loop.

<br>

#### `fix_names`
After running `missing_in_pops`, you'll notice that there are 11 countries that appear in `tots` that do not appear in `pops`. It turns out that most of these countries *do* appear in `pops`, just under a different name or spelling. Using the power of Google or your geographical knowledge, combined with an exploration of the `tots` DataFrame, complete the implementation of the function `fix_names` that takes in one argument – a DataFrame like `pops` – and returns a copy of `pops`, but with the `'Country (or dependency)'` column changed so that all countries that appear in `tots` also appear in the result, with a few exceptions listed below. 

For example, the country `'Burma'` appears in `tots`, but is listed as `'Myanmar'` in `pops`. In the DataFrame returned by your function, `'Myanmar'` should be changed to `'Burma'` to match `tots`.

As noted above, there are a few exceptions:

- `'Kosovo'` does not appear in `pops`. You do not need to change an entry in `pops` to match it.
- There is a "country" simply called `'World'`. This is obviously not a country, and doesn't appear in `pops`. You do not need to change an entry in `pops` to match it.

In [45]:
tots

,Doses_admin,People_at_least_one_dose
Country_Region,,
Afghanistan,1.244987e+07,1.145969e+07
Albania,3.025728e+06,1.342243e+06
Algeria,1.526744e+07,7.840131e+06
Andorra,1.543670e+05,5.790100e+04
Angola,2.370105e+07,1.499013e+07
...,...,...
West Bank and Gaza,3.748571e+06,2.012767e+06
World,1.319737e+10,5.503074e+09
Yemen,1.239152e+06,1.003208e+06


In [34]:
# run this to see the result, and don't change this cell -- it is used by the tests
missing = missing_in_pops(tots, pops)
missing

{'Burma',
 'Cabo Verde',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 "Cote d'Ivoire",
 'Czechia',
 'Korea, South',
 'Kosovo',
 'US',
 'West Bank and Gaza',
 'World'}

In [42]:
missing_in_pops(tots, fix_names(pops))

{'Kosovo', 'World'}

In [43]:
# run this to see the result, and don't change this cell -- it is used by the tests
pops_fixed = fix_names(pops)
pops_fixed

,Country (or dependency),Population in 2023,World Percentage,Area (Km²),Density (P/Km²),ISO
0,India,1428627663,0.1785,3287590,480.5033,IND
1,China,1425671352,0.1781,9706961,151.2696,CHN
2,US,339996563,0.0425,9372610,37.1686,USA
3,Indonesia,277534122,0.0347,1904569,147.8196,IDN
4,Pakistan,240485658,0.0300,881912,311.9625,PAK
...,...,...,...,...,...,...
200,San Marino,33642,0.0000,61,560.7000,SMR
201,Palau,18058,0.0000,459,39.2565,PLW
202,Nauru,12780,0.0000,21,639.0000,NRU
203,Tuvalu,11396,0.0000,26,379.8667,TUV


In [44]:
grader.check("q4")

q4 results: All test cases passed!

### Question 5

([return to the outline](#outline))

Now that we have all of our data in a clean, compatible format, it's time to address our original question.

Complete the implementation of the function `draw_choropleth`, which takes in a DataFrame like `tots`, and another DataFrame like `pops_fixed`, and returns a `plotly` figure object containing a [choropleth](https://en.wikipedia.org/wiki/Choropleth_map) of the **average number of doses per person for each country**. We define this quantity as:

$$
\text{Doses Per Person} = \frac{\text{Total Number of Doses Administered}}{\text{Total Population}}.
$$

To create your figure, you'll use the `px.choropleth` function and provide several arguments. This [`plotly` article](https://plotly.com/python/choropleth-maps/) will be extremely helpful.

Here are some requirements to get full credit for your choropleth:
- You'll need to set the `locations` appropriately to tell `plotly` which regions you'd like to plot. (You should not need to do much work for this; if you find yourself setting two arguments that contain "location" in the name, you will fail the autograder, so make sure to set `locations` only.)
- Make sure the legend is labelled `'Doses Per Person'` exactly.
- When we hover over a country, we should see the name of the `'Country (or dependency)'` in bold at the top, just like in the example below.
- Set the color scale to be something other than the default. Note that in this choropleth, you should use a sequential color scheme, which means that the intensity of the color assigned to a country is proportional to the value being plotted for that country (e.g. darker colors should correspond to more doses per person and lighter colors should correspond to fewer doses per person, or vice versa). Read more about the theory of sequential and diverging color schemes [here](https://blog.datawrapper.de/diverging-vs-sequential-color-scales/).
- Set the title of the plot appropriately; it's up to you what the title is, as long as it contains `'COVID Vaccine Doses Per Person'` somewhere.
- Make sure the entire world is visible in your plot, not just a certain region.
- **Only include countries for which we know both the total number of doses administered and the total population.**

An example plot that satisifes all of these conditions is shown below, though we encourage you to customize yours within the confines above. Can you change the font?

<img src='data/choropleth-example.png' width=70%>

Remember to return the figure object itself. That is, somewhere in your code you will have `fig = px.choropleth(...)`; make sure to also `return fig`.

In [61]:
set(fix_names(pops)['Country (or dependency)']) - set(tots.index)

{'Eritrea',
 'French Guiana',
 'Greenland',
 'Guadeloupe',
 'Hong Kong',
 'Martinique',
 'Mayotte',
 'North Korea',
 'Puerto Rico',
 'Reunion',
 'Turkmenistan',
 'Vatican City',
 'Western Sahara'}

In [57]:
tots

,Doses_admin,People_at_least_one_dose
Country_Region,,
Afghanistan,1.244987e+07,1.145969e+07
Albania,3.025728e+06,1.342243e+06
Algeria,1.526744e+07,7.840131e+06
Andorra,1.543670e+05,5.790100e+04
Angola,2.370105e+07,1.499013e+07
...,...,...
West Bank and Gaza,3.748571e+06,2.012767e+06
World,1.319737e+10,5.503074e+09
Yemen,1.239152e+06,1.003208e+06


Run the cell below to see your choropleth.

In [62]:
# run this to see the result, and don't change this cell -- it is used by the tests
fig = draw_choropleth(tots, pops_fixed)
fig.show()

In [63]:
grader.check("q5")

q5 results: All test cases passed!

That's it for Part 1 of the project! Did the average number of doses per person in any country surprise you?

***Note***: To export your plot to HTML, you can use `fig.write_html('path.html')`. This will give you a standalone file that contains your interactive plot. Try it out!

---

<a name='part2'></a>

## Part 2: Vaccine Effectiveness 💉

There has naturally been much interest in the effectiveness of COVID-19 vaccines. In particular, recent reports suggest that the effectiveness of vaccines – and the Pfizer vaccine in particular – may decrease over time. While this is true, the public perception of declining vaccine effectiveness has been affected by what is often a poor use of statistics by media outlets.

For example, take the following blurb from [this article](https://www.cbc.ca/news/world/israel-covid-delta-variant-booster-1.6159472) on the Canadian Broadcasting Corporation's website (emphasis added):

> Meanwhile, Israeli health officials reported what appeared to be a waning efficacy of the vaccine, including among those who had been double vaccinated. **Data showed that of the serious cases being admitted to hospital, around 60 per cent of patients were people who had been fully vaccinated**, though most were over 60 or with underlying health conditions.

This stat sounds terrible – if you're sick in a hospital bed with COVID-19, you're actually more likely to be vaccinated than not! If you're not a data scientist, you might come away with the conclusion that the vaccine is no longer effective against serious illness, or – even worse – that getting the vaccine will **increase** your risk of sickness. Of course, this statistic by itself is actually meaningless. For instance, imagine a world in which 100% of people are vaccinated. Then 100% of people admitted to the hospital will be vaccinated, too! In Question 8, we will explain the theory behind **vaccine effectiveness**, a statistic that more meaningfully measures the improvement in outcomes for vaccinated individuals over unvaccinated individuals.

To start, let's load in some data. For this project, we'll work with a dataset of vaccine effectiveness in Israel through mid-summer 2021. The vaccination and hospitalization records in the dataset are real. Israel was mostly vaccinated using the Pfizer vaccine, and was one of the first places where lots of data on the Delta variant was gathered.

In [64]:
# Download Israeli COVID vaccinations data from the ☁️
if not pathlib.Path(os.path.join('data', 'israel.csv')).exists():
    urllib.request.urlretrieve(
        'https://f000.backblazeb2.com/file/dsc-data/covid-israel/israel.csv',
        os.path.join('data', 'israel.csv')
    )

In [65]:
israel_raw = pd.read_csv(os.path.join('data', 'israel.csv'))
israel_raw

,Age,Vaccinated,Severe Sickness
0,23,1,0
1,60,1,0
2,43,1,0
3,40,1,0
4,71,1,0
...,...,...,...
6937537,14,1,0
6937538,30,1,0
6937539,18,0,0
6937540,45,1,0


Each row in the dataset is a person. Some are vaccinated (marked with a 1 in the `'Vaccinated'` column), and some became severely ill (marked with a 1 in the `'Severe Sickness'` column).

As always, we will first check that the data is clean.

### Question 6 (✅ Checkpoint Question)

<a name='Question-6'></a>

([return to the outline](#outline))

If you look at the data types of each column in `israel_raw`, you'll see that the `'Age'` column has a `dtype` of `object`. If you investigate further, you will see that a special value has been used to indicate that some ages are missing.

Complete the implementation of the function `clean_israel_data`, which takes in a DataFrame like `israel_raw` and returns a *new* DataFrame where the missing ages are replaced by `np.NaN`s and the `'Age'` column's `dtype` is `float`. Furthermore, the `'Vaccinated'` and `'Severe Sickness'` columns should be stored as `bool`s. The shape of the returned DataFrame should be the same as `israel_raw`, and, as usual, your function should not modify the input DataFrame.

In [66]:
# run this to see the result, and don't change this cell -- it is used by the tests
israel = clean_israel_data(israel_raw)
israel

,Age,Vaccinated,Severe Sickness
0,23.0,True,False
1,60.0,True,False
2,43.0,True,False
3,40.0,True,False
4,71.0,True,False
...,...,...,...
6937537,14.0,True,False
6937538,30.0,True,False
6937539,18.0,False,False
6937540,45.0,True,False


In [67]:
grader.check("q6")

q6 results: All test cases passed!

Note that by setting the `dtype`s of the latter two columns to `bool` instead of `int64`, we save a significant amount of memory:

In [68]:
israel_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6937542 entries, 0 to 6937541
Data columns (total 3 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   Age              object
 1   Vaccinated       int64 
 2   Severe Sickness  int64 
dtypes: int64(2), object(1)
memory usage: 158.8+ MB


In [69]:
israel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6937542 entries, 0 to 6937541
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Age              float64
 1   Vaccinated       bool   
 2   Severe Sickness  bool   
dtypes: bool(2), float64(1)
memory usage: 66.2 MB


We now need to decide what to do with the missing values in the `'Age'` column. Can we simply drop those rows? First, let's look at whether the missing values might be MCAR, NMAR, missing by design, etc. We'll first check to see if the distribution of the other columns differs depending on whether the ages are missing or not:

In [70]:
israel.assign(null_age=israel['Age'].isna()).groupby('null_age').mean()

,Age,Vaccinated,Severe Sickness
null_age,,,
False,41.036,0.812204,0.000074
True,NaN,0.805164,0.000000


### Question 7

([return to the outline](#outline))

Because the vaccination rate and severe sickness rate appear to be about the same whether or not `'Age'` is missing, we hypothesize that `'Age'` is MCAR (missing completely at random). Let's investigate.

#### `mcar_permutation_tests`

Complete the implementation of the function `mcar_permutation_tests` that takes in two arguments – a DataFrame like `israel` and a number `n_permutations` of permutations – and runs the two permutation tests described below. Your function should return a 2-tuple where the first entry is an array of the simulated test statistics for the first permutation test, and the second entry is an array of simulated test statistics for the second permutation test.

- The first permutation test should check the null hypothesis that values in the `'Vaccinated'` column for rows where the `'Age'` was missing were drawn from the same distribution as the values in rows where `'Age'` was not missing. The alternative hypothesis should be that they were drawn from *different* distributions. As your test statistic, use the **absolute difference in group means**. 
- The second permutation test should do the same, but for the `'Severe Sickness'` column.

***Note:*** Your code should run in less than two minutes when called with `n_permutations=100`. You should be able to run both permutation tests in a single `for`-loop.

<br>

#### `missingness_type`

Then, complete the implementation of the function `missingness_type` that takes no arguments and returns a single integer corresponding to the option below that you think describes the type of missingess in this data. You should use a significance level of 0.01 when making your decision:

1. MCAR (Missing completely at random)
2. MAR (Missing at random)
3. NMAR (Not missing at random)
4. Missing by design

In [76]:
# run this to see the result, and don't change this cell -- it is used by the tests
# expect this cell to take a while to run!
test_stats_vax, test_stats_sick = mcar_permutation_tests(israel, 100)

In [79]:
(test_stats_vax >= 0.00704).sum() / 100

0.06

In [80]:
(test_stats_sick >= 0.000074).sum() / 100

0.62

In [81]:
grader.check("q7")

q7 results: All test cases passed!

Let's proceed by simply dropping the rows with missing values:

In [84]:
vax = israel.dropna()

Moving forward, use `vax`, not `israel`. **Make sure to run the above cell even if you haven't completed Question 7!**

### Question 8 (✅ Checkpoint Question)

<a name='Question-8'></a>

([return to the outline](#outline))

***Note:*** Assume that "hospitalization" and "severe illness" mean the same thing; that is, if someone is hospitalized they developed severe illness and vice versa.

As mentioned at the start of Part 2, looking at the probability of being vaccinated if you're hospitalized is meaningless, since if everyone is vaccinated, this probability will be 1. Instead, we care about **the probability of being hospitalized if you're vaccinated.** More than that, what we care most about is how this probability relates to **the probability of being hospitalized if you're unvaccinated**.

To put it more concretely, let's say that 1 in 100 vaccinated people are hospitalized, while 10 in 100 unvaccinated people are hospitalized. This means that unvaccinated people are 10x more likely to be hospitalized than vaccinated people. To put it another way, if those 10 hospitalized unvaccinated people had been vaccinated, we'd expect that 9 of them would not have been hospitalized. That is, the vaccine would have prevented 90% of the hospitalizations, and we might therefore say that it is 90% effective against severe illness.

This is the intuition behind the definition of **vaccine effectiveness** (see the CDC's [page](https://www.cdc.gov/csels/dsepd/ss1978/lesson3/section6.html) for more information). That is, to compute a vaccine's effectiveness, we need to know two things:

1. The proportion of vaccinated people who developed severe illness (i.e., the probability that a vaccinated person will be hospitalized). Call this $p_V$.
2. The proportion of unvaccinated people who developed severe illness (i.e., the probability that an unvaccinated person will be hospitalized). Call this $p_U$.

Then:

$$
\text{effectiveness} = 1 - \frac{p_V}{p_U} = \frac{p_U - p_V}{p_U}
$$

**What this formula calculates, in words, is the proportion of unvaccinated people with severe illness who we expect would not have severe illness if they were vaccinated.** For instance, if no one who is vaccinated becomes severely ill, then $p_V = 0$ and the effectiveness is 1 (100%). On the other hand, if the vaccine is no better than being unvaccinated, $p_V = p_U$ and the effectiveness is 0 (0%).


Many news articles report effectiveness numbers, and this is certainly better than reporting the percentage of hospital patients that are vaccinated. Note, though, that there are different types of effectiveness – effectiveness against severe illness, effectiveness against symptomatic illness, effectiveness against death – and articles often are vague about which they are using. But even then, effectiveness can *still* be misleading due to statistical quirks, as we'll now see.


<br>

**Your job is to** complete the implementation of the function `effectiveness` that takes in a DataFrame like `vax` and returns the effectiveness of the vaccine against severe illness as a proportion between 0 and 1.


For example, if `df` is the DataFrame below, then run `effectiveness(df)` should evaluate to 0.5.


<table border="1" class="dataframe">  <thead>   <tr style="text-align: right;">      <th></th>      <th>Age</th>      <th>Vaccinated</th>      <th>Severe Sickness</th>    </tr>  </thead>  <tbody>   <tr>      <th>0</th>      <td>15</td>      <td>True</td>      <td>True</td>    </tr>    <tr>      <th>1</th>      <td>20</td>      <td>True</td>      <td>False</td>    </tr>    <tr>      <th>2</th>      <td>25</td>      <td>True</td>      <td>False</td>    </tr>    <tr>     <th>3</th>     <td>30</td>      <td>False</td>      <td>False</td>    </tr>    <tr>      <th>4</th>      <td>35</td>      <td>False</td>      <td>True</td>    </tr>    <tr>      <th>5</th>      <td>40</td>      <td>False</td>      <td>True</td>    </tr>  </tbody></table>


***Note:*** Assume that you will not run into any division-by-zero errors.

In [85]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_overall = effectiveness(vax)
eff_overall

0.6747820515830074

In [86]:
grader.check("q8")

q8 results: All test cases passed!

If you implemented `effectiveness` correctly, you should see a vaccine effectiveness between 60% and 70%. This is OK, but not great! It means that the vaccine is only around 60-70% effective at preventing severe illness overall.

But remember Simpson's paradox, which roughly says that sometimes a result that appears in aggregated data disappears when we look at disaggregated data. So let's disaggregate the data and perform the same calculation. That is, let's compute the effectiveness of the vaccine within each of several age groups. For convenience, here is a list of the age groups we'll consider:

In [87]:
AGE_GROUPS = [
    '12-15',
    '16-19',
    '20-29',
    '30-39',
    '40-49',
    '50-59',
    '60-69',
    '70-79',
    '80-89',
    '90-'
]

Each string above lists an age group. For example, the age group `'12-15'` should contain anyone aged 12, 13, 14, or 15. The last age group, '90-', denotes everyone who is 90 or above. `vax` does not contain anyone younger than 12.

### Question 9

([return to the outline](#outline))

Complete the implementation of the function `stratified_effectiveness`, that takes in one argument – a DataFrame like `vax` – and returns the effectiveness of the vaccine within each of the age groups in `AGE_GROUPS`. The return value of the function should be a Series of the same length as `AGE_GROUPS`, with the index of the Series being age groups as strings.

Don't use a `for`-loop!

In [88]:
vax

,Age,Vaccinated,Severe Sickness
0,23.0,True,False
1,60.0,True,False
2,43.0,True,False
3,40.0,True,False
4,71.0,True,False
...,...,...,...
6937537,14.0,True,False
6937538,30.0,True,False
6937539,18.0,False,False
6937540,45.0,True,False


In [98]:
# run this to see the result, and don't change this cell -- it is used by the tests
s_eff = stratified_effectiveness(vax)
s_eff

Group
12-15    1.000000
16-19    1.000000
20-29    1.000000
30-39    0.966591
40-49    0.941211
50-59    0.926886
60-69    0.886400
70-79    0.895772
80-89    0.810277
90-      0.924404
dtype: float64

In [100]:
grader.check("q9")

q9 results: All test cases passed!

If done correctly, you should notice that the effectiveness of the vaccine within each age group is higher than the overall effectiveness. In fact, the effectiveness of every age group is above 80%, and for most age groups it is above 90%.

This is an instance of [Simpson's paradox](https://dsc80.com/resources/lectures/lec08/lec08.html#Simpson's-paradox). This result might seem strange, or even impossible – how can *every one* of the age groups see a higher effectiveness rate than the overall effectiveness rate? We'll take a deeper look at this in the next question, but for now let's consider a slightly different scenario where we can give dosages of a vaccine on a scale, so that some people can receive more vaccine than others. We expect that the higher a dosage someone receives, the less sick they become. But suppose we plot the data and we see something like this:

<img src="./data/simpsons-aggregated.svg">

It actually looks like the more dosage someone receives, the more sick they get! But suppose we were to *disaggregate* the data into two age groups – old and young. That is, let's look at the same plot, but now mark young people as red and older people as purple. We might see this:

<img src="./data/simpsons.svg">

Now we see a different story: the larger the dosage, the less the sickness within each age group. Therefore, while the overall trend is a *positive* relationship between dosage and sickness, the within-group trends are all negative, as we'd hope. **The reason for this difference comes from the fact that, for each dosage level, the sickness rate of older people is greater than the sickness rate for younger people.**

If our data gives different answers depending on how we aggregate it, which answer should we use? In this case, we don't care about overall effectiveness – we care about how effective the vaccine will be for an *individual*. The data shows that, no matter what your age, unvaccinated people are ten times as likely to develop severe illness than vaccinated people, and it shows that the vaccine's effectiveness against severe illness is still quite strong.

The plots above may help to understand Simpson's paradox a little bit more, but it may still be unclear how each age group's effectiveness rate can be higher than the overall effectiveness rate. To explore this in more depth, **we'll build a calculator that takes in several arguments and calculates three effectivenesses: the overall (aggregate) effectiveness, the effectiveness among young people, and the effectiveness among old people.** For simplicity, we'll assume that there are just as many old people as young people.

To calculate these effectivenesses, you need to know six numbers:

1. The proportion of young people that are vaccinated.
2. The proportion of old people that are vaccinated.
3. The probability that a young vaccinated person will be hospitalized.
4. The probability that a young *un*vaccinated person will be hospitalized.
5. The probability that an old vaccinated person will be hospitalized.
6. The probability that an old *un*vaccinated person will be hospitalized.

Given these six pieces of information, you can calculate the overall effectiveness, the effectiveness in young people, and the effectiveness in old people.

How, you may ask? Well, to find the effectiveness in young people, you can use the effectiveness definition from [Question 8](#Question-8) ($1 - \frac{p_V}{p_U}$) directly since you are given the probability that a young vaccinated person will be hospitalized ($p_V$, for young people) and the probability that a young unvaccinated person will be hospitalized ($p_U$, for young people). The same applies for the effectiveness in old people.

To find the overall effectiveness, you need to find the values of $p_V$ and $p_U$ in the overall population, given  the six arguments that `effectiveness_calculator` takes in. 
- One approach is to express all six arguments in terms of conditional probability notation and to use your knowledge of probability theory from DSC 40A. One idea to look at in particular is the law of total probability ([Wikipedia article](https://en.wikipedia.org/wiki/Law_of_total_probability), [slides from DSC 40A](https://dsc-courses.github.io/dsc40a-2021-fa/resources/lecture/lec14-filled.pdf)).
- Another approach is to fix concrete numbers. 
    - For instance, assume that there are 1000 old people and 1000 young people (recall, we are operating under the assumption there are an equal number of old and young people, for simplicity).
    - To calculate the probability that a vaccinated person is hospitalized, you need to know the _number_ of vaccinated people that are hospitalized in this hypothetical example. If you're told that 20% of young people are vaccinated and that 10% of young vaccinated people will be hospitalized, you know that $1000 \cdot 0.2 \cdot 0.1 = 20$ young vaccinated people will be hospitalized. 
    - If you're also told that 60% of old people are vaccinated and that 30% of old vaccinated people will be hospitalized, then you know that $1000 \cdot 0.6 \cdot 0.3 = 180$ old vaccinated people will be hospitalized.
    - Using the above information, you know that there are $1000 \cdot 0.2 + 1000 \cdot 0.6 = 200 + 600 = 800$ vaccinated people total.
    - Putting this all together, 200 of the 800 vaccinated people will be hospitalized, so the overall $p_V$ is $p_V = \frac{200}{800} = 0.25$ in this example.
    - You can do the same for unvaccinated people to find the overall $p_U$.
    - Working with concrete numbers like this *is* using probability theory, just with extra steps. But it might be less abstract and easier to reason about.

### Question 10 (✅ Checkpoint Question)

<a name='Question-10'></a>

([return to the outline](#outline))

Complete the implementation of the function `effectiveness_calculator`, that takes in 6 arguments:

1. `young_vaccinated_prop`: The proportion of young people that are vaccinated.
2. `old_vaccinated_prop`: The proportion of old people that are vaccinated.
3. `young_risk_vaccinated`: The probability that a young vaccinated person will be hospitalized.
4. `young_risk_unvaccinated`: The probability that a young *un*vaccinated person will be hospitalized.
5. `old_risk_vaccinated`: The probability that an old vaccinated person will be hospitalized.
6. `old_risk_unvaccinated`: The probability that an old *un*vaccinated person will be hospitalized.

It should return a dictionary with three keys: `'Overall'`, `'Young'`, and `'Old'`, whose values are the overall effectiveness, the effectiveness within young people, and the effectiveness within old people, respectively. This will allow you to experiment with Simpson's paradox by plugging in different values and seeing what happens.

***Note:*** All arguments to `effectiveness_calculator` are numbers between 0 and 1 (inclusive).

***Hint:*** You are given two example inputs to `effectiveness_calculator` – one in the notebook below and one in the public test cases. Verify by hand that your code works correctly for **both** of them. 

In [49]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_example = effectiveness_calculator(
    young_vaccinated_prop=0.01,
    old_vaccinated_prop=0.99,
    young_risk_vaccinated=0.01,
    young_risk_unvaccinated=0.20,
    old_risk_vaccinated=0.10,
    old_risk_unvaccinated=0.50
)
eff_example

{'Young': 0.95, 'Old': 0.8, 'Overall': 0.5118226600985222}

In [50]:
grader.check("q10")

q10 results: All test cases passed!

It might be helpful to think about the results of your effectiveness calculator. In the example above, we have constructed an extreme situation where essentially all older people are vaccinated, but almost no younger people are. Therefore, when calculating the overall vaccine effectiveness, we're using the probability of illness given vaccination – which is essentially the probability of severe illness if you're old – and comparing it to the probability of severe illness if you're unvaccinated, which is essentially the probability of severe illness if you're young.

In fact, it is possible to create an example where the overall effectiveness is negative (taking the vaccine is worse than not taking it!), but where the effectiveness within groups is above 80%. The next question asks you to do just this. 

To help you with your experimentation, we've created an animated version of your `effectiveness_calculator`, that allows you to test different arguments by dragging sliders. Run the next cell to use it.

If you're curious, this sort of interactive tool is created using the `ipywidgets` package.

In [51]:
# The sliders below are in the same order as the arguments below,
# in case some of the slider labels are cut off.
interact(effectiveness_calculator,
                 young_vaccinated_prop=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.01),
                 old_vaccinated_prop=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.99),
                 young_risk_vaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.01),
                 young_risk_unvaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.20),
                 old_risk_vaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.10),
                 old_risk_unvaccinated=FloatSlider(min=0.01, max=0.99, step=0.01, value=0.50));

interactive(children=(FloatSlider(value=0.01, description='young_vaccinated_prop', max=0.99, min=0.01, step=0.…

### Question 11

([return to the outline](#outline))

Complete the implementation of the function `extreme_example`, which takes in no arguments and returns a dictionary whose keys are arguments to the function `effectiveness_calculator`. When your function is called and the dictionary is passed to `effectiveness_calculator`, it should return an `'Overall'` effectiveness that is negative and `'Young'` and `'Old'` effectivenesses that are both over 0.8.

***Note:*** Use the animated calculator above to help you, but **make sure to think about what the numbers mean**. There are many possible correct answers!

In [53]:
# run this to see the result, and don't change this cell -- it is used by the tests
eff_extreme = effectiveness_calculator(**extreme_example())
eff_extreme

{'Young': 0.95, 'Old': 0.8305084745762712, 'Overall': -0.11395101171459}

In [54]:
grader.check("q11")

q11 results: All test cases passed!

## Congratulations, you've finished Project 2! 🎉

As a reminder, all of the work you want to submit needs to be in `project.py` – this notebook should not be uploaded because there are no manually-graded questions in this project.

To verify that all of your work is indeed in `project.py`, and that you didn't accidentally implement a function in this notebook and not in `project.py`, we've included another notebook in the project folder, called `project-validation.ipynb`. `project-validation.ipynb` is a version of this notebook with only the `grader.check` cells and the code needed to set up the tests. 

### **Go to `project-validation.ipynb`, and go to Kernel > Restart & Run All.** This will check if all `grader.check` test cases pass using just the code in `project.py`.

Once you're able to pass all test cases in `project-validation.ipynb`, including the call to `grader.check_all()` at the very bottom, then you're ready to submit your `project.py` (and only your `project.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()